# McStas data to MJOLNIR data format, conversion script for BIFROST

by: Kristine M. L. Krighaar

created: 18/01/2024

In [12]:
from KGS import * # Utilizing my own package for data analysis. Remember to also send enviroment when sharing. 
from IPython.display import Image
import scipp as sc
import scippneutron as scn
import os

## Data reduction process

Full overview of how converison is done: 

![conversion_tree](DataReductionToMjolnir.png)

## Loading data from simulation

In [86]:
folder_path = "../McStasScript/result_folder_backend_mockup/backend_mockup_space/"
search_term = "detector"

file_list = [file for file in os.listdir(folder_path) if search_term in file]

I_A3 = []
Ierr_A3 = []

for i in range(2):
    print
    I = []
    I_err = []

    for file in file_list:

        # Extract data from simulation files 
        data = np.loadtxt(folder_path+file).reshape(3, 100, 100)
        I.append(data[0].T)
        I_err.append(data[1].T)

        print(i)

    I = np.asarray(I).T
    I_err = np.asarray(I_err).T
    
    I_A3.append(I)
    Ierr_A3.append(I_err)

I_A3 = np.asarray(I_A3)
Ierr_A3 = np.asarray(Ierr_A3)

print(np.shape(I_A3))


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
(2, 100, 100, 27)


## Organizing datastructure in scipp 

Datastructure includes lab information

In [70]:
data_dict = {}
for i in range(2):
    key = f'A3 setting{i}'
    value = sc.array(dims=['y', 't', 'tube'], values=I_A3[i])
    data_dict[key] = value

print(data_dict)

{'A3 setting0': <scipp.Variable> (y: 100, t: 100, tube: 27)    float64  [dimensionless]  [8.71221e-72, 0, ..., 0, 0], 'A3 setting1': <scipp.Variable> (y: 100, t: 100, tube: 27)    float64  [dimensionless]  [8.71221e-72, 0, ..., 0, 0]}


In [79]:
sim_data = sc.Dataset(
    data=data_dict, # sc.array(dims=['y', 't', 'tube'], values=I_A3),
    coords={
        'y': sc.array(dims=['y'], values=np.arange(100.0)),
        'A4': sc.array(dims=['y'], values=np.arange(100.0)),
        'Lsd': sc.array(dims=['y'], values=np.arange(100.0)),
        't': sc.array(dims=['t'], values=np.arange(100.0), unit='s'),
        'tube': sc.array(dims=['tube'], values=np.arange(27.0)),
        'Ana': sc.array(dims=['tube'], values=np.arange(27.0)),
        'wedge': sc.array(dims=['tube'], values=np.arange(27.0)),
    },
)

sc.show(sim_data)
sim_data


<scipp.Dataset>
Dimensions: Sizes[y:100, t:100, tube:27, ]
Coordinates:
* A4                        float64  [dimensionless]  (y)  [0, 1, ..., 98, 99]
* Ana                       float64  [dimensionless]  (tube)  [0, 1, ..., 25, 26]
* Lsd                       float64  [dimensionless]  (y)  [0, 1, ..., 98, 99]
* t                         float64              [s]  (t)  [0, 1, ..., 98, 99]
* tube                      float64  [dimensionless]  (tube)  [0, 1, ..., 25, 26]
* wedge                     float64  [dimensionless]  (tube)  [0, 1, ..., 25, 26]
* y                         float64  [dimensionless]  (y)  [0, 1, ..., 98, 99]
Data:
  A3 setting0               float64  [dimensionless]  (y, t, tube)  [8.71221e-72, 0, ..., 0, 0]
  A3 setting1               float64  [dimensionless]  (y, t, tube)  [8.71221e-72, 0, ..., 0, 0]